### Il est explicité ici l'utilisation des ensembles de codes écrits pour l'analyse de sensibilité sur des champs stochastiques et variables aléatoires.

### Il est d'abord conseillé d'installer l'environnement virtuel dont la définition se trouvent dans le fichier yaml

Nous allons faire ici l'analyse de sensiblité sur une poutre en flexion représentée par 100 éléments finis, et ou le module young et le diamètre de chaque élément est déterminé par un processus gaussien en une dimension. La position de la force, sa norme, tout comme la densité du matérieau sont déterminés par des lois nomales gaussiennes.
Dans la logique d'écriture de ces codes, il faut avoir un à priori sur les processus gaussiens, la loi qu'ils suivent, tout comme sur les paramètres des lois gaussiennes. Ensuite, il faut avoir une fonction python qui prend en entrée ces champs, et qui renvoie un ensemble connu de resultas. 


In [1]:
try:
    import anastruct, openturns, numba, joblib
except:
    import os
    if os.sys.platform == 'linux' :
        file_path = 'sensitivityEnv.yml'
        os.system('conda env create -f'+file_path)
        # to have the right modules installed
        print('now activate the environment and restart jupyter with other kernel')
    else :
        print('Do it alone')

In [2]:
# Voici les deux scripts destinés à gérer l'analyse de sensibilité sur les champs stochastiques
import NdGaussianProcessSensitivity as ngps
import NdGaussianProcessConstructor as ngpc
# Classes utilitaires
import numpy as np
import openturns as ot
# on importe aussi les fonctions à étudier
import RandomBeamGenerationClass    as rbgc

trend function args:  ['x']  trend function:  210000 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1
trend function args:  ['x']  trend function:  10 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1


D'abord, nous définisson l'ensemble de nos variables d'entrée, tout comme les variables de sortie :
- L'on va définir un par un tout les processus et variables aléatoires utilisées dans notre model. Bien sûr cela implique d'avoir un à-priori sur le comportement de ces différentes lois probabilistes. 
- Néanmoins, comme nous en sommes à des codes d'essai, il serait assez trivial de rajouter la prossiblité de récuperer l'approximation d'un champ inconnu avec l'approcimation de Karhunen - Loeve, en présence d'un grand nombre de mesures.
- La définition des éléments sur lesquels est construit le champ stochastique est un peu différent de la manière interne à openturns. En effet, s'étant placés directement dans un cadre de poutre en 'éléments finis, ou l'on a N+1 noeuds (N étant le nombre de poutres), il faut savoir si N est le nombre de mailles ou le nombre de noeuds. Dans le choix a été fait ici de définir la taille du maillage (grid_shape) de la manière suivante :
> grid_shape = [[position_X0, longeur_totaleX, nombre_mailles], [position_Y0, longeur_totaleY, nombre_mailles], ..]
 

In [3]:
# process governing the young modulus for each element      (MPa)
process_E = ngpc.NdGaussianProcessConstructor(dimension=1,
                                              grid_shape=[[0,1000,100],],
                                              covariance_model={'NameModel':'MaternModel',
                                                                'amplitude':5000.,
                                                                'scale':300,
                                                                'nu':13/3},
                                              trend_arguments=['x'],trend_function=210000)
process_E.setName('E_')

# process governing the diameter for each element          (mm)
process_D = ngpc.NdGaussianProcessConstructor(dimension=1,
                                              grid_shape=[[0,1000,100],],
                                              covariance_model={'NameModel':'MaternModel',
                                                                'amplitude':.3,
                                                                'scale':250,
                                                                'nu':7.4/3},
                                              trend_arguments=['x'],trend_function=10)
process_D.setName('D_')

# random variable for the density of the material (kg/m³)
rho         = 7850.
sigma       = 250
nameD       = 'Rho'
RV_Rho = ngpc.NormalDistribution(mu = rho, sigma = sigma, name = nameD)

# random variable for the position of the force   (mm) 
middle       = 500
sigma_f      = 50
namePos     = 'FP'
RV_Fpos = ngpc.NormalDistribution(mu = middle, sigma = sigma_f, name = namePos)

# random variable for the norm of the force    (N)
muForce       = 100
sigma_Fnor    = 15
nameNor       = 'FN'
RV_Fnorm = ngpc.NormalDistribution(mu = muForce, sigma = sigma_Fnor, name = nameNor)


trend function args:  ['x']  trend function:  210000 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1
trend function args:  ['x']  trend function:  10 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1


Il est important de noter que les processus et variables aléatoires doivent être définies à partir du module 
NdGaussianProcessConstructor, qui contient les classes suivantes : 
###### Classe pour construire un champ stochastique : 
> NdGaussianProcessConstructor.NdGaussianProcessConstructor() 
###### Classe pour construire une variables aléatoire  : 
> NdGaussianProcessConstructor.NormalDistribution() ## Classe openturns.Normal() suchargée
###### Classe pour construire un vecteur de variables aléatoires normales : 
> NdGaussianProcessConstructor.RandomNormalVector() ## Classe openturns.PythonRandomVector surchargée. Cette dernière est utilisée en interne par NdGaussianProcessConstructor.

**Finalement, l'autre particularité du NdGaussianProcessConstructor, est d'utiliser un objet _numpy.memmap_ modifiée, qui enregistre les échantillons de processus gaussiens sous forme de fichier temporaire dans le directoire d'utilisation des codes et les éfface en sortant du code**

L'on définit ensuite les variables de sortie. Il faut connaître l'ordre dans lequel la fonction renvoie ses resultats connaitre leur nom comme leur dimension. 
    

In [4]:
from importlib import reload
reload(ngps)
outputVariables = {'out1' :
                   {
                         'name'     : 'VonMisesStress',
                         'position' : 0,
                         'shape'    : (102,)  
                    },
                   'out2' :
                   {
                        'name'      : 'maxDeflection',
                        'position'  : 1,
                        'shape'     : (1,)
                   }
                  }
#Pour utiliser notre fonction, un wrapper a été spécialement écrit pour faciliter l'accès aux fonctions mais
#ce choix est entièrement dépendant de la manière ont a été définie la fonction sur laquelle vous travaillez.
functionWrapper = rbgc.sampleAndSoloFunctionWrapper(process_E, process_D, RV_Rho, RV_Fpos, RV_Fnorm)


###### Ensuite, on crée une instance pour l'analyse de sensibilité.
Il faut donner en entrée une liste contenant les procéssus et les variables aléatoires **dans l'ordre** dans lequel la fonction les recoit. 

*Pour cela il faut aussi connaître l'ordre des variables d'entrée de la fonction et leur nombre.*

**Ces fonctions sont celles qui prennent entrée des champs aléatoires (vecteurs et matrices, de type *list ou numpy*) et variables aléatoire (scalaires (vecteurs 1D pour le multiprocessing)) et non pas les fonctions prennant en entrée les variables aléatoires issues de la décomposition de la Karhunen Loeve, qui elles sont construites en interne dans la classe**

In [5]:
inputVarList = [process_E, process_D, RV_Rho, RV_Fpos, RV_Fnorm]
# We also the need the two functions of the model (one for samples, the other for single evaluations)
# In our case, as our model is defined as a class, we have to first create the model, 
# but it also could just be just two functions taking as an input the fields and RVs
soloFunction   = functionWrapper.randomBeamFunctionSolo
sampleFunction = functionWrapper.randomBeamFunctionSample
##
size           = 40 ## Number of samples for our sobol indicies experiment (kept low here to make things faster)
##
processSensitivityAnalysis = ngps.NdGaussianProcessSensitivityAnalysis(inputVarList, 
                                                                       outputVariables,
                                                                       sampleFunction,
                                                                       soloFunction,
                                                                       size)

trend function args:  ['x']  trend function:  210000 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1
trend function args:  ['x']  trend function:  10 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1


Voici une vue du dictionnaire intermédiaire qui est crée une fois les processus et variables aléatoires d'entrées définies :

On voit que la position des processus dans les arguments de la fonction d'entrée est enregistrée dans le dictionnaire dans la clé *position* et viennent de la manière de laquelle on a mis les VA et Processus dans le vecteur inputVarList. 


###### Ensuite, grace à l'intermédiaire de la classe ot.SobolIndiciesExperiment, on génère les' variables aléatoires d'entrée 

In [39]:
processSensitivityAnalysis.prepareSobolIndicesExperiment()


len(inputDesign) =  1040
input design is:         [ E_xi_0      E_xi_1      E_xi_2      ... Rho         FP          FN          ]
   0 : [   53.648      -1.98668    23.2094   ... 7877.33      487.673      88.2397   ]
   1 : [   55.7515     -0.912282   24.5215   ... 7997.63      435.915      77.6556   ]
   2 : [   55.0882      1.52042    24.7405   ... 8315.24      516.28      111.366    ]
...
1037 : [   51.3822      0.531884   25.2578   ... 7680.89      446.732     104.417    ]
1038 : [   55.872      -0.855109   24.9704   ... 7934.92      440.321      82.5617   ]
1039 : [   56.1749      0.892544   22.772    ... 8299.26      468.711     105.885    ]


In [53]:
reload(ngps)
inputDes = np.load('inputDesign.npy',allow_pickle=True)
inputDesNc = np.load('inputDesignNc.npy',allow_pickle=True)
processSensitivityAnalysis = ngps.NdGaussianProcessSensitivityAnalysis(inputVarList, 
                                                                       outputVariables,
                                                                       sampleFunction,
                                                                       soloFunction,
                                                                       size)
processSensitivityAnalysis.inputDesign = inputDes
processSensitivityAnalysis._inputDesignNC = inputDesNc

trend function args:  ['x']  trend function:  210000 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1
trend function args:  ['x']  trend function:  10 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1


###### Ensuite on récupère les sorties associées aux entrées génerées. 
L'idée du postprocessing est d'identifier si dans les sorties il y a des valeurs de type np.nan, et de refaire les experiences manquantes tant qu'il y a des np.nan de présents.

**Ceci crée quelques problèmes : En effet, si le calcul numérique n'a pas pu aboutir avec une certaine réalisation des variables en entrée, est-ce que cela veut dire que cette réalisation est défaillante? Va-t-on l'inclure dans le cas du calcul de la défaillance ou de la sensibilité? Car si on l'inclut pas, il se peut qu'on oublie un nombre conséquent de modes défaillants. Solution : changer le modèle informatique et le rendre robuste à ces erreurs, ou faire une étude précise avec ces réalisation particulières.**


La solution retenue pour génerer les experiences manquantes est d'itérer au dessus de chaque index de la sortie ou se trouvent les nans, de refaire une experience de sobol de taille 1 ( donc qui renverra d+2 valeurs, avec d la dimension de l'entrée), de générer les sorties correspondant aux entrées (recommencer si il s'y trouve un np.nan), et de remplacer les d+2 valeurs du inputDesign d'entrée avec celles que l'on vient de regénérer. 

In [54]:
processSensitivityAnalysis.getOutputDesignAndPostprocess()

shape deflection:  (1040, 103)  should be [N,10X] something
maxDeflection list:  [22.17614215 16.80989185 21.93583895 17.41561363 22.50603087 17.95891894
 18.73175937 23.28153639 26.45604793 22.76542661 19.65620801 25.0052461
 29.64726163 20.36505131 19.36147347 17.6870062  16.26091805 19.65929871
 13.06834139 20.89955123 17.16247955 21.05945664 22.91020409 22.52660179
 18.57490994 20.00832017 18.25249565 20.87327908 21.16395579 22.51000968
 15.45092384 19.07723803 22.49262591 18.1702287  16.70941053 15.60378439
 21.28061765 19.58798781 19.91709646 15.27863935 16.52542267 20.36816389]
deflection std deviation  nan
timed  96.689670086  s for function " randomBeamFunctionSample "
Converting list of outputs into matrix: 
Element  1  has shape  (1040, 102)
Element  2  has shape  (1040,)
Final shape matrix:  (1040, 103)
columns where nan :  [42]
There were  1  errors (numpy.nan) while processing, trying to regenerate missing outputs 

index to change:  [   2   42   82  122  162  202  242  2

Le *timing* vient de la classe ***custum_wraps***, qui est pour l'instant assez inutile, mais permet de prendre en main les décorateurs...

In [56]:
print(len(processSensitivityAnalysis.outputDesignList)) ## to be sure
output = processSensitivityAnalysis.outputDesignList ## We take the corrected output design
processSensitivityAnalysis.inputDesign

2


class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=1040 dimension=24 description=[E_xi_0,E_xi_1,E_xi_2,...,Rho,FP,FN] data=[[54.5451,0.0500083,23.1373,...,7891.89,461.472,104.375],[53.9228,1.86609,23.2743,...,7728.34,588.694,91.9991],[54.2009,-0.376983,24.9016,...,7728.6,439.612,111.612],...,[54.2416,-0.163182,22.6761,...,7897.96,557.713,107.783],[54.0852,-3.25802,24.0718,...,8126.09,527.67,123.663],[55.0325,-1.9978,25.6686,...,8382.75,531.34,75.2402]]

In [57]:
output2 = processSensitivityAnalysis._outputDesignListNC
print(processSensitivityAnalysis.inputDesign)
print(processSensitivityAnalysis.outputDesignList[1][42,...])
uu=ot.SaltelliSensitivityAlgorithm(inputDes,np.expand_dims(output[1], 1), size)
print(len(inputDes))
inputArr = np.array(inputDes)
np.argwhere(np.isnan(output[1]))
output[1][42]

       [ E_xi_0       E_xi_1       E_xi_2       ... Rho          FP           FN           ]
   0 : [   54.5451       0.0500083   23.1373    ... 7891.89       461.472      104.375     ]
   1 : [   53.9228       1.86609     23.2743    ... 7728.34       588.694       91.9991    ]
   2 : [   54.2009      -0.376983    24.9016    ... 7728.6        439.612      111.612     ]
...
1037 : [   54.2416      -0.163182    22.6761    ... 7897.96       557.713      107.783     ]
1038 : [   54.0852      -3.25802     24.0718    ... 8126.09       527.67       123.663     ]
1039 : [   55.0325      -1.9978      25.6686    ... 8382.75       531.34        75.2402    ]
3.8257519378494487
1040


3.8257519378494487

In [58]:
test = processSensitivityAnalysis.getSobolIndiciesKLCoefs()
import matplotlib.pyplot as plt
test[0][0,2].getFirstOrderIndices()

new shape is:  102
sobol field shape:  [1, 102]
Shape sensitivity field :  (1, 102)
new shape is:  1
[5.32174173 5.80617237 5.27570373 ... 6.15864666 7.09829272 3.70079264]


class=Point name=Unnamed dimension=24 values=[-0.0965959,-0.0966484,-0.0965763,-0.0966216,-0.0966157,-0.0966263,-0.0966127,-0.0966767,-0.0626493,0.0437696,-0.0494265,-0.0910594,-0.133882,-0.0752132,-0.0830312,-0.0982919,-0.0968424,-0.0930179,-0.0936057,-0.0967187,-0.0977158,-0.0966767,0.280053,0.782144]

In [50]:
inputArray = np.array(processSensitivityAnalysis.inputDesign)
inputArrayNc = np.array(processSensitivityAnalysis._inputDesignNC)
outputArray = processSensitivityAnalysis.outputDesignList[0]
outputArrayNc = processSensitivityAnalysis._outputDesignListNC[0]
print(np.array_equal(inputArray, inputArrayNc))
print(np.array_equal(outputArray, outputArrayNc))
print(outputArray[43])
print(outputArrayNc[43])


False
False
[  1.74255148   6.17303613  11.8903633   17.67049123  23.49888823
  29.38239247  35.31138731  41.25849752  47.1847756   53.048955
  58.8174411   64.47281592  70.01929805  75.48413992  80.91513412
  86.37499059  91.93376928  97.66056056 103.61551779 109.84280963
 116.36520093 123.18065661 130.26159376 137.55701273 144.99773848
 152.5045597  159.99821791 167.40988188 174.69051824 181.81714062
 188.79521826 195.65673697 202.45423997 209.25233836 216.11796977
 223.11067274 230.27444668 237.63191962 245.18119061 252.89609094
 260.72920856 268.61777104 276.49143768 284.28074331 291.92528363
 297.43900688 297.36119331 292.61143035 287.61497793 282.40808202
 277.03353032 271.53371535 265.94501212 260.29436591 254.59808488
 248.86299341 243.08917305 237.27407301 231.41649613 225.52013171
 219.59580689 213.66189637 207.74330229 201.86861431 196.06645136
 190.3614563  184.77069072 179.30082675 173.94702394 168.69306007
 163.5131772  158.3752344  153.24482968 148.08965317 142.88390229


In [51]:
print(np.argwhere(np.isnan(output)))
print(min(outputList[0]))
print(max(outputList[0]))
print(processSensitivityAnalysis.inputDesign)
print(np.expand_dims(output[...,50], 1))

ValueError: could not broadcast input array from shape (1040,102) into shape (1040)

In [52]:
inputDesign  = processSensitivityAnalysis.inputDesign
inputList    = np.asarray(inputDesign).tolist()
print(len(inputList))
print(len(outputList[10]))
sensitivityAnalysisList = [ot.SaltelliSensitivityAlgorithm(inputDesign, np.expand_dims(output[...,i], 1), size) for i in range(len(outputList))]

TypeError: object of type 'NoneType' has no len()

In [ ]:
sensitivityAnalysisList[0].getFirstOrderIndices()

In [ ]:
import matplotlib.pyplot as plt
sobolIndiciesList = [sensitivityAnalysisList[i].getFirstOrderIndices() for i in range(len(sensitivityAnalysisList))]
sobolIndiciesArr  = np.asarray(sobolIndiciesList)
print(sobolIndiciesArr.shape)
plt.imshow(sobolIndiciesArr.T)

In [ ]:
help(inputDesign.add)


In [ ]:
inputDesign[0] = np.arange(24).tolist()
inputDesign[]


In [ ]:
len(outputVariables.keys())


In [ ]:
x=functionWrapper.results[1]
print(x.shape)
x1 = x[:,1,:]
print(x1.shape)